# Анализ базы данных сервиса для чтения книг по подписке

**Цель исследования**: извлечь и проанализировать данные о книгах, издательствах, авторах, а также пользовательских обзорах книг из базы данных купленного сервиса для чтения книг по подписке и разработать предложения для нового продукта.

**Исследовательские вопросы:**
- Сколько книг вышло после 1 января 2000 года?
- Сколько для каждой книги обзоров и какая средняя оценка?
- Какое издательство выпустило наибольшее число книг толще 50 страниц?
- Кто автор книг с самой высокой средней оценкой (учитываютя книги с 50 и более оценками)?
- Каково среднее количество обзоров от пользователей, которые поставили больше 50 оценок?

## Загрузка  и обзор данных

Загрузим необходимые библиотеки.

In [14]:
import pandas as pd
from sqlalchemy import create_engine
# некоторые коды по загрузке данных удалены, т.к. базы данных являются интеллектуальной собственностью Практикума

Устанавливаем параметры.

Посмотрим на первые 5 строк всех таблиц.

In [4]:
books = '''
SELECT * FROM books LIMIT 5
'''
select(books)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [5]:
authors = '''
SELECT * FROM authors LIMIT 5
'''
select(authors)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [6]:
publishers = '''
SELECT * FROM publishers LIMIT 5
'''
select(publishers)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [7]:
ratings = '''
SELECT * FROM ratings LIMIT 5
'''
select(ratings)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [8]:
reviews = '''
SELECT * FROM reviews LIMIT 5
'''
select(reviews)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Исследовательские вопросы

**Количество книг, выпущенных после 1 января 2000 года**

In [9]:
books_count = '''
SELECT COUNT(book_id) AS books_cnt FROM books WHERE publication_date >= '2000-01-02'
'''
select(books_count)

,books_cnt
0,819


После 1 января 2000 года выпущено 819 книг.

**Количество обзоров и средняя оценка для каждой книги**

In [10]:
ratings_reviews = '''
SELECT pivot_reviews.title, reviews_count, avg_rating
            FROM
            (SELECT books.book_id, books.title, COUNT(DISTINCT reviews.review_id) AS reviews_count FROM books
            LEFT JOIN reviews ON reviews.book_id=books.book_id
            GROUP BY books.book_id, books.title) AS pivot_reviews
            JOIN
            (SELECT books.book_id, books.title, AVG(ratings.rating) AS avg_rating FROM books
            JOIN ratings ON ratings.book_id=books.book_id
            GROUP BY books.book_id, books.title
            ORDER BY 1) AS pivot_ratings
            ON pivot_reviews.book_id = pivot_ratings.book_id
            ORDER BY 2 DESC
'''
select(ratings_reviews)

,title,reviews_count,avg_rating
0,Twilight (Twilight #1),7,3.662500
1,Water for Elephants,6,3.977273
2,The Glass Castle,6,4.206897
3,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,The Natural Way to Draw,0,3.000000
997,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,Essential Tales and Poems,0,4.000000


Согласно таблице, пользователи мало пишут обзоров на прочитанные книги (не более 7 обзоров на книгу). Необходимо узнать причину такой ситуации, вероятно, есть какие-то технические проблемы с написанием отзывов, если таковых нет, следует как-то мотивировать пользователей  к написанию отзывов (дать какие-то бонусы, специальные предложения и т.п.), т.к. с точки зрения развития сервиса 
с точки зрения рекомендательной системы для новых читателей, важно, чтобы были отзывы, поскольку они заинтересовывают новых читателей  и могут побудить к покупке книг.

**Издательство, которое выпустило наибольшее число книг толще 50 страниц**

In [11]:
book_publisher = '''
SELECT
     publishers.publisher AS name_publisher,
     COUNT(books.book_id) AS cnt_of_published_books
FROM
    publishers
INNER JOIN books ON books.publisher_id = publishers.publisher_id
WHERE
    books.num_pages > 50
GROUP BY
    name_publisher
ORDER BY 
    cnt_of_published_books DESC
LIMIT 5
'''
select(book_publisher)

,name_publisher,cnt_of_published_books
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19


Наибольшее число книг, толще 50 стр. выпустило издательство Penguin Books.

**Автор книг с самой высокой средней оценкой (учитываютя книги с 50 и более оценками)**


In [12]:
best_author = '''
SELECT
      SUBQ.author AS author,
      ROUND(AVG(SUBQ.avg_rating),2) AS avg_rating
FROM
(SELECT
     authors.author AS author,
     books.book_id AS id,
     AVG(ratings.rating) AS avg_rating,
     COUNT(ratings.rating) AS cnt_rating
FROM
    authors
INNER JOIN books ON books.author_id = authors.author_id
INNER JOIN ratings ON ratings.book_id = books.book_id
GROUP BY
    author,
    id
    HAVING
    COUNT(ratings.rating) >= 50) AS SUBQ
GROUP BY
    author
ORDER BY
    avg_rating DESC
LIMIT 5    
'''
select(best_author)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.28
1,Markus Zusak/Cao Xuân Việt Khương,4.26
2,J.R.R. Tolkien,4.26
3,Louisa May Alcott,4.19
4,Rick Riordan,4.08


Автором с самой высокой средней оценкой (4,28) является британская писательница Джоан Роулинг (J.K. Rowling).

**Среднее количество обзоров от пользователей, которые поставили больше 50 оценок**

In [13]:
avg_review = '''
SELECT ROUND(avg(count)) 
FROM
(SELECT
    COUNT(review_id)
FROM 
    reviews
FULL OUTER JOIN (SELECT username,
                COUNT(rating_id) as cnt
FROM 
    ratings
GROUP BY
        username) AS cnt_rating
ON cnt_rating.username = reviews.username
where cnt > 50
GROUP BY reviews.username) as count;                                  
'''
select(avg_review)

,round
0,24.0


Пользователи, поставившие более 50 оценок в среднем пишут 24 обзора, таким образом, можно сделать вывод, что чем больше читатели ставят оценки, тем больше делают обзоров.

## Общие выводы 

В хода анализа мы ответили на все исследовательские вопросы:
- 819 книг  выпущено после 1 января 2000 года.
- Читатели пишут мало обзоров на прочитанные книги, максимальное количество обзоров на книгу равно 7.
- Издательство Penguin Books выпустило наибольшее число книг, толще 50 стр. 
- Автором с самой высокой средней оценкой (4,28) является Джоан Роулинг.
- Пользователи, поставившие более 50 оценок в среднем пишут 24 обзора.

Результаты данного анализа могут быть использованы при развитии данного приложения.